Goal: use Panadas Dataframe Merge to join many related tables into a single table

In [1]:
import arcpy
import time
import pandas as pd

In [25]:
# Poles use shape@ to create point feature for damage assessment
PoleFldsOrig = ['GLOBALID','SUBTYPE','OWNER','FACILITYID','POLESIZE','MATERIAL','COLOR','TREATMENTTYPE','YEARMANUFACTURED','STUBDATE','BILLINGGROUP','FEEDERIDS','INSTALL_NUM','UNIT_TYPE','STREETADDRESS','BACKBONEINDICATOR','StockNumber','SHAPE@']
PoleFldsDest = ['eSupportStructure_GLOBALID','eSupportStructure_SUBTYPE','eSupportStructure_OWNER','eSupportStructure_FACILITYID','eSupportStructure_POLESIZE','eSupportStructure_MATERIAL','eSupportStructure_COLOR','eSupportStructure_TREATMENTTYPE','eSupportStructure_YEARMANUFACTURED','eSupportStructure_STUBDATE','eSupportStructure_BILLINGGROUP','eSupportStructure_FEEDERIDS','eSupportStructure_INSTALLNUM','eSupportStructure_UNITTYPE','eSupportStructure_STREETADDRESS','eSupportStructure_BACKBONE','eSupportStructure_StockNumber','SHAPE@']
PoleFldsDestOut = ['eSupportStructure_GLOBALID','eSupportStructure_SUBTYPE','eSupportStructure_OWNER','eSupportStructure_FACILITYID','eSupportStructure_POLESIZE','eSupportStructure_MATERIAL','eSupportStructure_COLOR','eSupportStructure_TREATMENTTYPE','eSupportStructure_YEARMANUFACTURED','eSupportStructure_STUBDATE','eSupportStructure_BILLINGGROUP','eSupportStructure_FEEDERIDS','eSupportStructure_INSTALLNUM','eSupportStructure_UNITTYPE','eSupportStructure_STREETADDRESS','eSupportStructure_BACKBONE','eSupportStructure_StockNumber']

# feature classes that have 1:1 relationship with poles
CapBankFldsOrig = ['eSupportStructure_GLOBALID','DEVICEID','SUBTYPE','STRUCTUREID','GLOBALID']
CapBankFldsRename = {'DEVICEID':'eCapacitorBank_DEVICEID','SUBTYPE':'eCapacitorBank_SUBTYPE','STRUCTUREID':'eCapacitorBank_STRUCTUREID','GLOBALID':'eCapacitorBank_GLOBALID'}
CapBankFldsDest = ['eSupportStructure_GLOBALID','eCapacitorBank_DEVICEID','eCapacitorBank_SUBTYPE','eCapacitorBank_STRUCTUREID','eCapacitorBank_GLOBALID']

RecloserFldsOrig = ['eSupportStructure_GLOBALID','DEVICEID','FEEDERID','FEEDERID2','PHASEDESIGNATION','SUBTYPE','AMPRATING','BACKBONEPASSTHRU','CUSTOMERCOUNT','STRUCTUREID','INTERRUPTINGTYPE','INTERRUPTRATING','CONTROLLERTYPE','BACKBONEINDICATOR','GLOBALID']
RecloserFldsRename = {'DEVICEID':'eRecloser_DEVICEID','FEEDERID':'eRecloser_FEEDERID','FEEDERID2':'eRecloser_FEEDERID2','PHASEDESIGNATION':'eRecloser_PHASE','SUBTYPE':'eRecloser_SUBTYPE','AMPRATING':'eRecloser_AMPRATING','BACKBONEPASSTHRU':'eRecloser_BACKBONEPASSTHRU','CUSTOMERCOUNT':'eRecloser_CUSTOMERCOUNT','STRUCTUREID':'eRecloser_STRUCTUREID','INTERRUPTINGTYPE':'eRecloser_INTERRUPTINGTYPE','INTERRUPTRATING':'eRecloser_INTERRUPTRATING','CONTROLLERTYPE':'eRecloser_CONTROLLERTYPE','BACKBONEINDICATOR':'eRecloser_BACKBONEINDICATOR','GLOBALID':'eRecloser_GLOBALID'}
RecloserFldsDest = ['eSupportStructure_GLOBALID','eRecloser_DEVICEID','eRecloser_FEEDERID','eRecloser_FEEDERID2','eRecloser_PHASE','eRecloser_SUBTYPE','eRecloser_AMPRATING','eRecloser_BACKBONEPASSTHRU','eRecloser_CUSTOMERCOUNT','eRecloser_STRUCTUREID','eRecloser_INTERRUPTINGTYPE','eRecloser_INTERRUPTRATING','eRecloser_CONTROLLERTYPE','eRecloser_BACKBONE','eRecloser_GLOBALID']

SectionalizerFldsOrig = ['eSupportStructure_GLOBALID','DEVICEID','FEEDERID','FEEDERID2','BACKBONEPASSTHRU','CUSTOMERCOUNT','STRUCTUREID','GLOBALID']
SectionalizerFldsDest = ['eSupportStructure_GLOBALID','eSectionalizer_DEVICEID','eSectionalizer_FEEDERID','eSectionalizer_FEEDERID2','eSectionalize_BACKBONEPASSTHRU','eSectionalizer_CUSTOMERCOUNT','eSectionalizer_STRUCTUREID','eSectionalizer_GLOBALID']
VoltRegBankFldsOrig = ['eSupportStructure_GLOBALID','DEVICEID','FEEDERID','STRUCTUREID','GLOBALID']
VoltRegBankFldsDest = ['eSupportStructure_GLOBALID','eVoltRegBank_DEVICEID','eVoltRegBank_FEEDERID','eVoltRegBank_STRUCTUREID','eVoltRegBank_GLOBALID']


In [11]:
# fGDB variables
unitTable = r'C:\Users\friendde\Documents\ArcGIS\Projects\PandasDataframeMerge\PandasDataframeMerge.gdb\MIMS\mmPoleEquipment'
connectionRoot = r'C:\Users\friendde\Documents\ArcGIS\Projects\PandasDataframeMerge\PandasDataframeMerge.gdb\%s'
workspace = r'C:\Users\friendde\Documents\ArcGIS\Projects\PandasDataframeMerge\PandasDataframeMerge.gdb'
origFGDB = r'C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.'

# feature classes
MM_POLEEQUIPMENT_XML = r"C:\Users\friendde\Documents\ArcGIS\Projects\PandasDataframeMerge\MIMS_DATASET_20181019.XML"
poleFC = r'C:\GISData\Data\Snapshot\mxElectric.geodatabase\Electric\main.eSupportStructure'
origFC = ['eCapacitorBank','eRecloser','eSectionalizer','eVoltageRegulatorBank']
fuseFC = 'eFuse'
lightFC = 'eLight'
#poleFC = 'eSupportStructure'
servicepointFC = 'eServicePoint'
switchFC = 'eSwitch'
transformerFC = 'eTransformerBank'

In [12]:
# prep unitTable
if not arcpy.Exists(unitTable):
    arcpy.ImportXMLWorkspaceDocument_management(workspace, MM_POLEEQUIPMENT_XML, "Schema_Only")
else:
    print('Truncating...',unitTable)
    arcpy.TruncateTable_management(unitTable)

Truncating... C:\Users\friendde\Documents\ArcGIS\Projects\PandasDataframeMerge\PandasDataframeMerge.gdb\MIMS\mmPoleEquipment


In [13]:
# Start Main
with arcpy.da.Editor(workspace) as edit:
    print('Inserting...poles')
    ic = arcpy.da.InsertCursor(unitTable,PoleFldsDest)
    with arcpy.da.SearchCursor(poleFC,PoleFldsOrig) as sc:
        for scrow in sc:
            ic.insertRow(scrow)
    del ic
    del sc

Inserting...poles


In [29]:
nparrUnitTable = arcpy.da.TableToNumPyArray(unitTable,PoleFldsDestOut,skip_nulls=True)
dfUnitTable = pd.DataFrame(nparrUnitTable)
#pdarr.to_csv(custAcctFile,header=False, index=False)
dfUnitTable

,eSupportStructure_GLOBALID,eSupportStructure_SUBTYPE,eSupportStructure_OWNER,eSupportStructure_FACILITYID,eSupportStructure_POLESIZE,eSupportStructure_MATERIAL,eSupportStructure_COLOR,eSupportStructure_TREATMENTTYPE,eSupportStructure_YEARMANUFACTURED,eSupportStructure_STUBDATE,eSupportStructure_BILLINGGROUP,eSupportStructure_FEEDERIDS,eSupportStructure_INSTALLNUM,eSupportStructure_UNITTYPE,eSupportStructure_STREETADDRESS,eSupportStructure_BACKBONE,eSupportStructure_StockNumber
0,{E355591F-61E2-4BB9-9FC2-F42377076DC8},29,GRU,82143,035/,WOOD,NONE,CREOS,9999,2013-10-23,RRU,"289,",5000100703,I-R_P12,"1901 NE 9TH ST, GAINESVILLE",N,773069
1,{062B2C22-8A31-4DA4-8B81-AEF3CAA48887},29,GRU,26061,035/5,WOOD,NONE,CCA,9999,2012-02-17,APU,"205,",5000191293,I-S_P12,"2860 SE 19TH AVE, GAINESVILLE",N,773069
2,{BBD0FEA0-804D-4955-B535-23C93AB39F88},29,GRU,11561,040/,WOOD,NONE,CREOS,0,2016-04-29,CPU,"1031,",5000191288,I-S_P15,"3855 NW 17TH AVE, GAINESVILLE",N,317241
3,{1DC17E60-F6A7-4569-B23A-79B2C1C7C2E1},29,GRU,83242,040/5,WOOD,NONE,CREOS,1974,2016-09-12,RRU,"934,",5000196399,I-R_P15,"3230 SW ARCHER RD, I144, GAINESVILLE",N,317241
4,{9FE5E088-E573-41BA-9524-66BA8A6BF130},19,GRU,36661,040/,WOOD,NONE,NONE,9999,2014-12-03,CPC,"936,",5000191350,I-S_P15,"2925 SW 40TH BLVD, GAINESVILLE",N,317241
5,{154BA316-C104-4298-9301-24BFA56AD670},29,GRU,23957,040/,WOOD,NONE,CREOS,1968,2014-10-01,CPU,"1033,",5000191288,I-S_P15,"5611 W NEWBERRY RD, GAINESVILLE",N,317241
6,{500754C5-CE23-497B-8BA0-C612642FFB25},29,GRU,23929,045/3,WOOD,NONE,CREOS,9999,2016-07-06,CPU,"1033,",5000191288,I-S_P18,"5500 W NEWBERRY RD, GAINESVILLE",N,188336
7,{D972A421-432E-4DEC-9CF6-568C7D9CFA21},29,GRU,29445,030/4,WOOD,NONE,PENTA,9999,2010-11-05,CPU,"282,",5000191288,I-S_P08,"500 E UNIVERSITY AVE, GAINESVILLE",N,570117


In [30]:
nparrCapBank = arcpy.da.TableToNumPyArray(r'C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eCapacitorBank',CapBankFldsOrig,skip_nulls=True)
dfCapBank = pd.DataFrame(nparrCapBank)
#pdarr.to_csv(custAcctFile,header=False, index=False)
dfCapBank

,eSupportStructure_GLOBALID,DEVICEID,SUBTYPE,STRUCTUREID,GLOBALID
0,{A4421322-3B40-45DA-A750-9EA416910103},CAP669,1,34448,{CD48648C-72FF-439E-90BC-DF7802A7D66B}
1,{AE53004E-2461-4653-B750-3550E68C8EE3},CAP461,1,24557,{CA3E31B4-A598-4565-A2AD-70DEF82E267B}
2,{8539C40C-3B37-47F2-B01A-A6545FFF2B11},CAP406,1,30932,{56A17C1F-F0D5-412F-B699-6D256A947E56}
3,{F1334E84-D39B-4972-BFF6-63E90875D68A},CAP230,1,90139,{3B4AD031-3D3C-4FD0-9998-7B3671309E58}
4,{8DBA2386-DCB3-4EB6-B522-16D57E31EA35},CAP760,1,39800,{6FFEC47B-8F5C-42C7-9E6A-8E8D2FAFDA42}
5,{03866385-C296-4D10-B47B-121AA08034CC},CAP408,1,35419,{09F7BD24-F7D9-4D44-BC56-610F7A2B9636}
6,{1C45D0DD-FD7A-4770-BF8C-3746E5D51107},CAP642,1,38332,{F8A7A16A-98D6-4C25-948C-FB6DF9731644}
7,{03779E56-8DBD-4CB8-9F9E-929E4B924210},CAP739,1,48805,{B9B6EFDE-1BC7-4BEB-A083-37954EBE9D99}
8,{8CCEA48A-D124-4F2C-B118-F85E0D786AAC},CAP944,1,40720,{2B53C88F-8720-4DA8-8628-8EF953A8047A}
9,{5F51D3F3-D78D-467F-A8E2-352359B68659},CAP729,1,43351,{D9A68F2B-7BD8-428D-9FAB-EB2256B68104}


DataFrame.merge(right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html

In [31]:
df2 = dfUnitTable.merge(dfCapBank, how='outer', on='eSupportStructure_GLOBALID', left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=False, indicator=False, validate=None)

In [32]:
df2.rename(index=str, columns=CapBankFldsRename)

,eSupportStructure_GLOBALID,eSupportStructure_SUBTYPE,eSupportStructure_OWNER,eSupportStructure_FACILITYID,eSupportStructure_POLESIZE,eSupportStructure_MATERIAL,eSupportStructure_COLOR,eSupportStructure_TREATMENTTYPE,eSupportStructure_YEARMANUFACTURED,eSupportStructure_STUBDATE,...,eSupportStructure_FEEDERIDS,eSupportStructure_INSTALLNUM,eSupportStructure_UNITTYPE,eSupportStructure_STREETADDRESS,eSupportStructure_BACKBONE,eSupportStructure_StockNumber,eCapacitorBank_DEVICEID,eCapacitorBank_SUBTYPE,eCapacitorBank_STRUCTUREID,eCapacitorBank_GLOBALID
0,{E355591F-61E2-4BB9-9FC2-F42377076DC8},29.0,GRU,82143,035/,WOOD,NONE,CREOS,9999.0,2013-10-23,...,"289,",5000100703,I-R_P12,"1901 NE 9TH ST, GAINESVILLE",N,773069,NaN,NaN,NaN,NaN
1,{062B2C22-8A31-4DA4-8B81-AEF3CAA48887},29.0,GRU,26061,035/5,WOOD,NONE,CCA,9999.0,2012-02-17,...,"205,",5000191293,I-S_P12,"2860 SE 19TH AVE, GAINESVILLE",N,773069,NaN,NaN,NaN,NaN
2,{BBD0FEA0-804D-4955-B535-23C93AB39F88},29.0,GRU,11561,040/,WOOD,NONE,CREOS,0.0,2016-04-29,...,"1031,",5000191288,I-S_P15,"3855 NW 17TH AVE, GAINESVILLE",N,317241,NaN,NaN,NaN,NaN
3,{1DC17E60-F6A7-4569-B23A-79B2C1C7C2E1},29.0,GRU,83242,040/5,WOOD,NONE,CREOS,1974.0,2016-09-12,...,"934,",5000196399,I-R_P15,"3230 SW ARCHER RD, I144, GAINESVILLE",N,317241,NaN,NaN,NaN,NaN
4,{9FE5E088-E573-41BA-9524-66BA8A6BF130},19.0,GRU,36661,040/,WOOD,NONE,NONE,9999.0,2014-12-03,...,"936,",5000191350,I-S_P15,"2925 SW 40TH BLVD, GAINESVILLE",N,317241,NaN,NaN,NaN,NaN
5,{154BA316-C104-4298-9301-24BFA56AD670},29.0,GRU,23957,040/,WOOD,NONE,CREOS,1968.0,2014-10-01,...,"1033,",5000191288,I-S_P15,"5611 W NEWBERRY RD, GAINESVILLE",N,317241,NaN,NaN,NaN,NaN
6,{500754C5-CE23-497B-8BA0-C612642FFB25},29.0,GRU,23929,045/3,WOOD,NONE,CREOS,9999.0,2016-07-06,...,"1033,",5000191288,I-S_P18,"5500 W NEWBERRY RD, GAINESVILLE",N,188336,NaN,NaN,NaN,NaN
7,{D972A421-432E-4DEC-9CF6-568C7D9CFA21},29.0,GRU,29445,030/4,WOOD,NONE,PENTA,9999.0,2010-11-05,...,"282,",5000191288,I-S_P08,"500 E UNIVERSITY AVE, GAINESVILLE",N,570117,NaN,NaN,NaN,NaN
8,{A4421322-3B40-45DA-A750-9EA416910103},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,CAP669,1.0,34448,{CD48648C-72FF-439E-90BC-DF7802A7D66B}
9,{AE53004E-2461-4653-B750-3550E68C8EE3},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,CAP461,1.0,24557,{CA3E31B4-A598-4565-A2AD-70DEF82E267B}


In [33]:
nparrRecloser = arcpy.da.TableToNumPyArray(r'C:\GISData\Data\Snapshot\mxElectric.geodatabase\main.eRecloser',RecloserFldsOrig,skip_nulls=True)
dfRecloser = pd.DataFrame(nparrRecloser)
#pdarr.to_csv(custAcctFile,header=False, index=False)
dfRecloser

,eSupportStructure_GLOBALID,DEVICEID,FEEDERID,FEEDERID2,PHASEDESIGNATION,SUBTYPE,AMPRATING,BACKBONEPASSTHRU,CUSTOMERCOUNT,STRUCTUREID,INTERRUPTINGTYPE,INTERRUPTRATING,CONTROLLERTYPE,BACKBONEINDICATOR,GLOBALID
0,{DB6B5903-8878-4E60-805D-9C702E26FC5C},MOS36H,1029,1032,7,3,630,Y,0,22875,Solid,12500,RMT,Y,{A743239C-7C6C-4B15-BE95-691FC8EC5CB7}
1,{304EBB5D-ABBA-4F5A-8EA6-FF3C82B709ED},MOS94K,215,435,7,3,630,Y,0,21296,Solid,12500,MAN,Y,{41498DCE-9651-42FE-B2FF-E3A78762D00D}
2,{15D83386-B2E5-49EA-8CF6-3A9F45DF1E91},MOS347,1433,208,7,3,630,Y,0,37661,Solid,12500,RMT,Y,{5A2453A5-9D8E-4F61-A4D4-4927B3381CF3}
3,{DFC5A63C-41F4-436C-87C9-4604588C6C25},MOS84W,216,286,7,3,630,Y,0,54616,Solid,12500,MAN,Y,{B5F1EFAE-1134-483F-A796-018F9E300396}
4,{D4D4AAE8-0826-4096-AF2C-E3B049D7A94E},MOS274,433,434,7,3,630,Y,0,35092,Solid,12500,MAN,Y,{153FD475-07F7-4C75-AFE4-8F95E0FFC2E2}
5,{DB25B9D9-1F5B-45C4-9408-46095BDF1A34},MOS209,1534,550,7,3,630,Y,0,31158,Solid,12500,MAN,Y,{FE648965-0CE5-4FF8-84ED-3D62B2D4B0D7}
6,{ABB65CEF-1605-42A2-AC71-827F2BE834C8},MOS55M,211,541,7,3,630,Y,0,10747,Solid,12500,MAN,Y,{45A45760-9E90-48D5-90F3-1D17991C8F25}
7,{E1A3E3E4-18F3-481F-ABF4-7CE4FCB98E7F},MOS61N,1033,1037,7,3,630,Y,0,37545,Solid,12500,MAN,Y,{72B68640-D7CA-485C-AE39-D81D771155BB}
8,{5F1536A4-B833-4AAF-A174-23C8C1643568},MOS477,539,544,7,3,630,Y,0,35729,Solid,12500,MAN,Y,{534F3027-8957-4D2C-99E9-B0F3980BAEEC}
9,{C72D0A54-55FD-4CD2-A174-13D598648E82},MOS233,546,831,7,3,630,Y,0,20642,Solid,12500,RMT,Y,{6B2769A4-3C5F-4A41-8D31-0B688CED18C7}


In [34]:
df4 = df3.merge(dfRecloser, how='outer', on='eSupportStructure_GLOBALID', left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=False, indicator=False, validate=None)

In [35]:
df4.rename(index=str, columns=RecloserFldsRename)

,eSupportStructure_GLOBALID,eSupportStructure_SUBTYPE,eSupportStructure_OWNER,eSupportStructure_FACILITYID,eSupportStructure_POLESIZE,eSupportStructure_MATERIAL,eSupportStructure_COLOR,eSupportStructure_TREATMENTTYPE,eSupportStructure_YEARMANUFACTURED,eSupportStructure_STUBDATE,...,eRecloser_SUBTYPE,AMPRATING_y,BACKBONEPASSTHRU_y,CUSTOMERCOUNT_y,eRecloser_STRUCTUREID,INTERRUPTINGTYPE_y,INTERRUPTRATING_y,CONTROLLERTYPE_y,BACKBONEINDICATOR_y,eRecloser_GLOBALID
0,{E355591F-61E2-4BB9-9FC2-F42377076DC8},29.0,GRU,82143,035/,WOOD,NONE,CREOS,9999.0,2013-10-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,{062B2C22-8A31-4DA4-8B81-AEF3CAA48887},29.0,GRU,26061,035/5,WOOD,NONE,CCA,9999.0,2012-02-17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,{BBD0FEA0-804D-4955-B535-23C93AB39F88},29.0,GRU,11561,040/,WOOD,NONE,CREOS,0.0,2016-04-29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{1DC17E60-F6A7-4569-B23A-79B2C1C7C2E1},29.0,GRU,83242,040/5,WOOD,NONE,CREOS,1974.0,2016-09-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,{9FE5E088-E573-41BA-9524-66BA8A6BF130},19.0,GRU,36661,040/,WOOD,NONE,NONE,9999.0,2014-12-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,{154BA316-C104-4298-9301-24BFA56AD670},29.0,GRU,23957,040/,WOOD,NONE,CREOS,1968.0,2014-10-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,{500754C5-CE23-497B-8BA0-C612642FFB25},29.0,GRU,23929,045/3,WOOD,NONE,CREOS,9999.0,2016-07-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,{D972A421-432E-4DEC-9CF6-568C7D9CFA21},29.0,GRU,29445,030/4,WOOD,NONE,PENTA,9999.0,2010-11-05,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,{A4421322-3B40-45DA-A750-9EA416910103},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,{AE53004E-2461-4653-B750-3550E68C8EE3},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
